# Attention

言語モデルを学ぶ上では欠かせないAttentionという機構について。

In [ ]:
import math
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [ ]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
textfile_ja = "data/iwslt2017_ja.txt"
textfile_en = "data/iwslt2017_en.txt"
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"

with open(textfile_en) as f:
    data_en = f.read().splitlines()

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()

n_data = len(data_en)
print("num of data:", n_data)

sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")
unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of data: 223108
num of vocabrary (ja): 8000
num of vocabrary (en): 8000


In [ ]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_ja.append(eos_id)
    ids_en.insert(0, bos_id)
    ids_en.append(eos_id)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja
        x_dec = en[:-1]
        y_dec = en[1:]
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch):
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

num of train data: 178487
num of test data: 44621



---

## Attention

複数のデータの中から重要なデータに着目する仕組み。Attention = 注意、注目、着目。

前章で作成したSeq2Seqによる翻訳モデルでは、入力文をEncoderによって固定長のベクトルに変換し、それをDecoderに渡すことで、入力文に基づいた文章を生成した。

EncoderがRNNであるとき、Encoderは各時刻で隠れ状態を出力できる。この中から最後の時間の隠れ状態のみをDecoderに渡していたものがこれまでのSeq2Seqである。このとき、encoderが出力する全ての隠れ状態を利用したいと考える。その方が入力文の多くの情報を参照でき、より適切な出力が得られそうだ。そして、それを実現する仕組みがAttentionである。



複数のデータの中から重要なデータに注目する仕組み。AttentionをSeq2Seqに取り入れることで、Decoderが、Encoderが出力した情報の中の重要な情報に注目するようになる。

前章で作成したSeq2Seqによる翻訳モデルでは、入力文をEncoderによって固定長のベクトルに変換し、それをDecoderに渡すことで、入力文に基づいた文章を生成した。

EncoderがRNNであるとき、Encoderは各時刻で隠れ状態を出力できる。この中から最後の時間の隠れ状態のみをDecoderに渡していたものがこれまでのSeq2Seqである。このとき、encoderが出力する全ての隠れ状態を利用したいと考える。その方が入力文の多くの情報を参照でき、より適切な出力が得られそうだ。そして、それを実現する仕組みがAttentionである。

隠れ状態の数は入力の系列長に依る。例によって可変長のデータはNNでは扱いづらく、Seq2Seqでは「最後の時刻の隠れ状態」とすることで固定長のベクトルを得ていた。

- encoderは全ての時刻で隠れ状態を出力する
- 

- encoderは各時刻でhを出力する
- hの数は入力の系列長に依存する
- 可変長のデータはNNで扱いづらいので固定長のベクトルが欲しい
- ここで候補として最後の隠れ状態や全ての隠れ状態の平均が挙がった
- 結局最後のhのみを使おう、となったのがこれまでのSeq2Seq
- ここで
    - 全ての隠れ状態の平均を使うことを考える
- decoderの時刻によって参照するベクトルが変わって欲しい
- 


---

## Attention機構

複数のデータの中から重要なデータに着目する仕組み。Attention = 注意、注目、着目。

ある一つの入力と、関連する複数のデータを考える。関連する複数のデータはmemoryと呼ぶ。入力を元に、memoryの中のどのデータに着目するかを定めることがAttentionの目的である。各データに重要度を割り当てるという感じ。

重要度は重みと呼ばれ、$w_i$で表すことにする。重みは総和が1になるようにsoftmaxなどで正規化する。正規化前の値はスコアと呼んだりする。

各memoryに対応する重みは入力との内積で求める。別に内積じゃなくてもいいけど、内積が一番簡単だし性能も良い。内積が取れるように、memoryの各ベクトルは入力と同じ次元にする必要がある。

重みを求めた後は、その重みでmemoryの重み付き和をとる。そうすることで、memoryの中から重要な要素を多めに取り出した固定長のベクトルが得られる。

やってみよう。まず、入力と、三つのデータからなるmemoryを用意する。

In [6]:
n, d = 3, 5

x = torch.randn(d)
memory = torch.randn(n, d)
memory

tensor([[ 0.7003,  0.3728,  0.9630, -0.7548,  1.4186],
        [-0.3834,  0.7128, -0.5010, -0.5049,  0.4977],
        [-0.0481,  0.5967,  2.3803,  0.1163,  0.1539]])

入力とmemory内の全てのデータで内積を取る。これがスコアに当たる。

In [7]:
scores = torch.tensor([m @ x for m in memory])
scores

tensor([0.3264, 0.1925, 0.4811])

これでもいい。

In [ ]:
scores = x @ memory.T
scores

softmaxで正規化する。

In [ ]:
weights = F.softmax(scores, dim=-1)
weights

この重みがmemoryの各データの重要度を表す。これで重み付き和をとる。

In [8]:
# 重みをかける
weighted_memory = torch.stack([w * m for w, m in zip(weights, memory)])
weighted_memory

tensor([[ 0.2286,  0.1217,  0.3143, -0.2464,  0.4630],
        [-0.0738,  0.1372, -0.0964, -0.0972,  0.0958],
        [-0.0231,  0.2871,  1.1452,  0.0560,  0.0740]])

In [9]:
# 和をとる
y = weighted_memory.sum(dim=0)
y

tensor([ 0.1316,  0.5460,  1.3631, -0.2876,  0.6329])

以下のようにまとめられる。

In [10]:
y = weights @ memory # 重み付き和
y

tensor([ 0.1316,  0.5460,  1.3631, -0.2876,  0.6329])

以上がattention機構の演算の流れである。まとめるとこう。

In [11]:
scores = memory @ x # スコア
weights = F.softmax(scores, dim=-1) # 重み
y = weights @ memory # 重み付き和
y

tensor([ 0.1316,  0.5460,  1.3631, -0.2876,  0.6329])

数式だとこうなる。

$$
\text{Attention}(\boldsymbol x,M) = \text{softmax}(\boldsymbol xM^T)M
$$

- $\boldsymbol x\in\R^{d}$ : 入力
- $M\in\R^{n\times d}$ : memory

\*列ベクトルと行ベクトルを区別していないので厳密ではない。厳密に書くならこう:

$$
\text{Attention}(\boldsymbol x,M) = (\text{softmax}(\boldsymbol x^TM^T)M)^T
$$


---

## Attentionを用いたSeq2Seq

Seq2SeqのDecoderにこのAttentionを導入し、Encoderが出力した全ての隠れ状態を参照する。

現状、Encoderの隠れ状態は最後のもののみがDecoderに渡っているが、可能であれば全ての隠れ状態をDecoderに渡したい。その方が多くの情報を参照できそう。しかし、Encoderの隠れ状態の数はEncoderへの入力の数によって変化する。可変長のデータは通常のNNでは扱いづらいため、最後の時刻の隠れ状態と入力定長のベクトルを渡していた。

ここでAttentionを用いる。Decoderのある時刻の隠れ状態を入力、Encoderの全ての隠れ状態をmemoryとしてAttentionの演算を行う。こうすることで、Decoderは各時刻にて、Encoderの全ての隠れ状態から注目すべき重要な情報を都合よく抽出した固定長のベクトルを得ることが出来る。つまりEncoderで固定長のベクトルを用意する必要がなくなる。

In [12]:
seq_len = 3
hidden_size = 5
hs_enc = torch.randn(seq_len, hidden_size) # encoderが出力した全ての隠れ状態
h_dec = torch.randn(hidden_size) # ある時間tのdecoderの隠れ状態

scores = h_dec @ hs_enc.T # (seq_len,)
weights = F.softmax(scores, dim=-1)
y = weights @ hs_enc # (hidden_size,)
y # encoderの全ての隠れ状態から重要な部分を多く抜き出したベクトル

tensor([ 0.4589,  0.3038,  1.8257, -0.6385,  0.2316])

ちなみに、重みが正しく着目すべき点を表すかは、学習させてみないと分からない。この目的も、学習前の段階では期待に過ぎない。この仕組みを取り入れて学習させれば、次第に適切な重みが出力されるようになり、適切な出力が得られるようになるだろう。そうだといいな、ってだけ。

全ての隠れ状態を参照した固定長のベクトルを得るだけであれば、単に全ての隠れ状態を足すだけでもいい。ただ、重みを変えられるような枠組みを取り入れてあげれば学習が上手くいくんじゃね？ってだけ。そして本当にうまくいったからここで紹介されている。

また、重みを求める関数が内積でないといけない理由はない。2つのベクトルからスカラーを得る関数であれば何でもよい。

内積は類似度を測ることができ、類似度が高いものに着目するという意味では適切に見えるが、そもそも比較するベクトルはいくつかの層を経て複雑に変化するため、それらの類似度は意味を持たない。重みを求める関数を内積として学習を進めれば、重要度が高くなるべきタイミングでその2つのベクトルが類似するように学習される、というだけ。

ただ実際はほとんどの場合で内積が使われる。それは内積という計算がシンプルだからってだけ。

### Attention層

Decoderの中の、Attentionによって都合のいい隠れ状態を出力する部分は一つの層として見られる。複数時刻の入力を考慮して以下のように表す。

$$
\text{Attention}(X,M) = \text{softmax}(XM^T)M
$$

- $X\in\R^{n_i\times d}$ : 層への入力
- $M\in\R^{n_m\times d}$ : memory

実装してみよう。

In [13]:
class Attention(nn.Module):
    def forward(self, x, hs):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h

In [14]:
batch_size, seq_len_dec, seq_len_enc, hidden_size = 2, 3, 4, 5
x = torch.randn(batch_size, seq_len_dec, hidden_size)
hs = torch.randn(batch_size, seq_len_enc, hidden_size)

attention = Attention()
h = attention(x, hs)
h.shape

torch.Size([2, 3, 5])

### MASK

padトークンがAttentionの計算に含まれてしまうことを回避する。maskをかけてpadトークンに対応する重みが0になるようにする。

スコアに対して、対応する位置の値を$-\infty$にする。そうすればsoftmaxを計算したときにその部分が0になる。

$$
\text{Attention}(\boldsymbol x,M) = \text{softmax}(\boldsymbol xM^T-\infty\,\text{mask}) M
$$

こんなスコアがあったとする。

In [15]:
scores = torch.randn(5)
scores

tensor([ 1.7205, -0.7592,  1.8133, -0.2011, -0.0768])

後ろの2つがpadトークンだったとすると、こんな感じでmaskをかけてやればいい。

In [16]:
mask = [False, False, False, True, True]
scores[mask] = -torch.inf
scores

tensor([ 1.7205, -0.7592,  1.8133,    -inf,    -inf])

こう書いてもいい。

In [17]:
scores = torch.randn(5)
mask = torch.tensor([0, 0, 0, 1, 1])
scores.masked_fill_(mask, -torch.inf)
scores

tensor([ 0.2911, -1.3570, -0.8464,    -inf,    -inf])

後はこれをsoftmaxに通す。

In [18]:
weights = F.softmax(scores, dim=-1)
weights

tensor([0.6609, 0.1272, 0.2119, 0.0000, 0.0000])

できた。これでpadトークンが無視されるようになる。

層としても実装する。

In [19]:
class Attention(nn.Module):
    def forward(self, x, hs, mask=None):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        mask: (batch_size, seq_len_enc), bool, padトークンの位置
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        if mask is not None:
            scores.masked_fill_(mask.unsqueeze(1), -torch.inf) # maskを適用
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h


---

## 実践

実際にAttentionをSeqSeqに取り入れて翻訳モデルを学習させてみる。

### モデル構築

まずEncoder。全ての時刻の隠れ状態を出力する。

In [ ]:
# class Encoder(nn.Module):
#     def __init__(
#         self,
#         n_vocab,
#         embed_size,
#         hidden_size,
#         dropout=0.2,
#     ):
#         super().__init__()
#         self.embedding = nn.Embedding(n_vocab, embed_size)
#         self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, hidden_size)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         """
#         x: (batch_size, seq_len)
#         """
#         eos_pos = x == eos_id
#             # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
#         x = self.embedding(x) # (batch_size, seq_len, embed_size)
#         hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
#         h = hs[eos_pos] # (batch_size, hidden_size)
#         h = self.dropout(h)
#         h = self.fc(h) # (batch_size, hidden_size)
#         return h

次にDecoder。~~~

In [ ]:
# class Decoder(nn.Module):
#     def __init__(
#         self,
#         n_vocab,
#         embed_size,
#         hidden_size,
#         dropout=0.2,
#     ):
#         super().__init__()
#         self.embedding = nn.Embedding(n_vocab, embed_size)
#         self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
#         self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
#         self.attention = Attention()
#         self.fc = nn.Linear(hidden_size, n_vocab)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, hc):
#         x = self.embedding(x) # (batch_size, seq_len, embed_size)
#         hs, hc = self.lstm1(x, hc) # (batch_size, seq_len, hidden_size)
        
#         hs = self.dropout(hs)
#         y = self.fc(hs) # (batch_size, seq_len, n_vocab)
#         return y, hc

最後に、これらをまとめる。

全ての隠れ状態とpadトークンの位置をDecoderに渡すようにする。

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        hs = self.encoder(x_enc)
        mask = x_enc == pad_id
        y, _ = self.decoder(x_dec, hs, mask=mask)
        return y

In [ ]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 35,926,336


### 学習

In [28]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [40]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train(model, optimizer, n_epochs=20, prog_unit=1)

### 翻訳

In [26]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str,
    max_len: int = 100,
    decisive: bool = True,
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor([in_ids + [eos_id]], device=device)

    hs, h = model.encoder(in_ids)
    hc = (h, torch.zeros_like(h))
    next_token = bos_id

    token_ids = []
    while len(token_ids) < max_len and next_token != eos_id:
        x = torch.tensor([[next_token]], device=device)
        y, hc = model.decoder(x, hs, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

In [27]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 彼にセンスのない詩を書いたり セーターを編んだりしました
output: And I was writing terrible poetry and knitting sweaters for him.
answer: And I was writing terrible poetry and knitting sweaters for him.

input: その後 2分間に 3人が2才のワン・ユーの側を通り過ぎます
output: Within two minutes, three people pass two-year-old Wang Yue by.
answer: Within two minutes, three people pass two-year-old Wang Yue by.

input: もう1つはNoksha-Yug Acceesで、 農村地域の物流を統合することを目的に マイクロファイナンスに基づく自助組織財団から基金を受けました。
output: And the other is Moksha-Yug Access, which is integrating rural supply chain on the foundations of self-help group-based microfinance.
answer: And the other is Moksha-Yug Access, which is integrating rural supply chain on the foundations of self-help group-based microfinance.

input: この小さな機器で何でもします
output: I do so many things on this little device.
answer: I do so many things on this little device.

input: 我々は暴力には力で対峙し 混沌には混沌で対峙しました
output: We met violence with force and chaos with chaos.
answer: We met violence with force and chaos with c

In [1]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

NameError: name 'n' is not defined

In [29]:
# original
sentences = [
    "ありがとう。",
    "猫はかわいいね。",
    "上手く文章が書けるようになりました。"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう。
output: Thank you.

input: 猫はかわいいね。
output: I'm here to do a magic.

input: 上手く文章が書けるようになりました。
output: I wanted to race that. I wanted to look at this time, and I was now going to make it all the first time in the  ⁇ 



びみょう。


---

## Attentionの可視化

